In [3]:
# Importing Packages and Data
import pandas as pd
import numpy as np
import biogeme as biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
from biogeme.expressions import Beta
from biogeme.expressions import (
    Beta,
    DefineVariable,
    bioDraws,
    PanelLikelihoodTrajectory,
    MonteCarlo,
    log,
    Derive,
    bioNormalCdf,
    Elem
)
import math
from datetime import datetime
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer
import biogeme.biogeme as bio
from biogeme.expressions import Beta, log, Elem, bioNormalCdf

In [4]:
# Import psychometric variables
micro_psych = pd.read_csv('Data/Psych_Micro.csv')
# drop unique_ID column
micro_psych = micro_psych.drop('unique_id', axis=1)
micro_psych = micro_psych.drop('user', axis = 1)
# drop envbenefit4 since unused and contains NaNs
micro_psych = micro_psych.drop('envbenefit4', axis = 1)


In [18]:
# sort
micro_psych = micro_psych.sort_values(by=['who'])

# convert to database
database = db.Database('micro_psych', micro_psych)

# database.panel("who")


globals().update(database.variables)

In [19]:
# parameters to be estimated

intercept_SN = Beta('intercept_SN', 0, None, None, 0)
coef_veh_SN = Beta('coef_veh_SN', 0, None, None, 0)
coef_bike_SN = Beta('coef_bike_SN', 0, None, None, 0)
coef_white_SN = Beta('coef_white_SN', 0, None, None, 0)
coef_higheredu_SN = Beta('coef_higheredu_SN', 0, None, None, 0)
coef_employ_SN = Beta('coef_employ_SN', 0, None, None, 0)

intercept_EP = Beta('intercept_EP', 0, None, None, 0)
coef_veh_EP = Beta('coef_veh_EP', 0, None, None, 0)
coef_bike_EP = Beta('coef_bike_EP', 0, None, None, 0)
coef_stu_EP = Beta('coef_stu_EP', 0, None, None, 0)
coef_higheredu_EP = Beta('coef_higheredu_EP', 0, None, None, 0)

# intercept_RISK = Beta('intercept_RISK', 0, None, None, 0)
# coef_bike_RISK = Beta('coef_bike_RISK', 0, None, None, 0)
# coef_stu_RISK = Beta('coef_stu_RISK', 0, None, None, 0)
# coef_gender_RISK = Beta('coef_gender_RISK', 0, None, None, 0)
# coef_white_RISK = Beta('coef_white_RISK', 0, None, None, 0)
# coef_higheredu_RISK = Beta('coef_higheredu_RISK', 0, None, None, 0)
# coef_employ_RISK = Beta('coef_employ_RISK', 0, None, None, 0)

intercept_EB = Beta('intercept_EB', 0, None, None, 0)
coef_veh_EB = Beta('coef_veh_EB', 0, None, None, 0)
coef_bike_EB = Beta('coef_bike_EB', 0, None, None, 0)
coef_gender_EB = Beta('coef_gender_EB', 0, None, None, 0)

coef_SN = Beta('coef_SN', 0, None, None, 0)
coef_EP = Beta('coef_EP', 0, None, None, 0)
coef_RISK = Beta('coef_RISK', 0, None, None, 0)
coef_EB = Beta('coef_EB', 0, None, None, 0)


In [27]:
# define latent variables
# STRUCTURAL EQUATIONS

SN = (
    intercept_SN
    + coef_veh_SN * veh_dummy
    + coef_bike_SN * bike
    + coef_white_SN * white
    + coef_higheredu_SN * higheredu
    + coef_employ_SN * employ
)

EP = (
    intercept_EP
    + coef_veh_EP * veh_dummy
    + coef_bike_EP * bike
    + coef_stu_EP * stu_dummy
    + coef_higheredu_EP * higheredu
)

# RISK = (
#     intercept_RISK
#     + coef_bike_RISK * bike 
#     + coef_stu_RISK * stu_dummy
#     + coef_gender_RISK * gender_F
#     + coef_white_RISK * white
#     + coef_higheredu_RISK * higheredu
#     + coef_employ_RISK * employ
#     # + sigma_ATT * omega_ATT
# )

EB = (
    intercept_EB
    + coef_veh_EB * veh
    + coef_bike_EB * bike
    + coef_gender_EB * gender_F
)

# INTENT = (
#     + coef_SN*SN
#     + coef_EP * EP
#     + coef_RISK * RISK
#     + coef_EB * EB
# )


In [28]:
B_SN01_F1 = Beta('B_SN01_F1',-1,None,None,1)
B_SN02_F1 = Beta('B_SN02_F1',-1,None,None,0)
B_SN03_F1 = Beta('B_SN03_F1',-1,None,None,0)

INTER_SN01 = Beta('INTER_SN01',0,None,None,1)
INTER_SN02 = Beta('INTER_SN02',0,None,None,0)
INTER_SN03 = Beta('INTER_SN03',0,None,None,0)

MODEL_SN01 = INTER_SN01 + B_SN01_F1 * SN
MODEL_SN02 = INTER_SN02 + B_SN02_F1 * SN
MODEL_SN03 = INTER_SN03 + B_SN03_F1 * SN

SIGMA_STAR_SN01 = Beta('SIGMA_STAR_SN01',1,1.0e-5,None,0)
SIGMA_STAR_SN02 = Beta('SIGMA_STAR_SN02',1,1.0e-5,None,0)
SIGMA_STAR_SN03 = Beta('SIGMA_STAR_SN03',1,1.0e-5,None,0)

delta_1 = Beta('delta_1',0.1, 1.0e-5, None, 1)
delta_2 = Beta('delta_2',0.2,1.0e-5, None, 0)
tau_1 = -delta_1 - delta_2
tau_2 = -delta_1
tau_3 = 0
tau_4 = delta_1
tau_5 = delta_1 + delta_2

SN01_tau_1 = (tau_1-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_2 = (tau_2-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_3 = (tau_3-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_4 = (tau_4-MODEL_SN01)/SIGMA_STAR_SN01
SN01_tau_5 = (tau_5-MODEL_SN01)/SIGMA_STAR_SN01

SN02_tau_1 = (tau_1-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_2 = (tau_2-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_3 = (tau_3-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_4 = (tau_4-MODEL_SN02)/SIGMA_STAR_SN02
SN02_tau_5 = (tau_5-MODEL_SN02)/SIGMA_STAR_SN02

SN03_tau_1 = (tau_1-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_2 = (tau_2-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_3 = (tau_3-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_4 = (tau_4-MODEL_SN03)/SIGMA_STAR_SN03
SN03_tau_5 = (tau_5-MODEL_SN03)/SIGMA_STAR_SN03

# Likert scale with M = 6 measurements
IndSN01 = {
    1: bioNormalCdf(SN01_tau_1),
    2: bioNormalCdf(SN01_tau_2)-bioNormalCdf(SN01_tau_1),
    3: bioNormalCdf(SN01_tau_3)-bioNormalCdf(SN01_tau_2),
    4: bioNormalCdf(SN01_tau_4)-bioNormalCdf(SN01_tau_3),
    5: bioNormalCdf(SN01_tau_5)-bioNormalCdf(SN01_tau_4),
    6: 1-bioNormalCdf(SN01_tau_5)
}
P_SN01 = Elem(IndSN01, socialnorm1)


IndSN02 = {
    1: bioNormalCdf(SN02_tau_1),
    2: bioNormalCdf(SN02_tau_2)-bioNormalCdf(SN02_tau_1),
    3: bioNormalCdf(SN02_tau_3)-bioNormalCdf(SN02_tau_2),
    4: bioNormalCdf(SN02_tau_4)-bioNormalCdf(SN02_tau_3),
    5: bioNormalCdf(SN02_tau_5)-bioNormalCdf(SN02_tau_4),
    6: 1-bioNormalCdf(SN02_tau_5)
}
P_SN02 = Elem(IndSN02, socialnorm2)

IndSN03 = {
    1: bioNormalCdf(SN03_tau_1),
    2: bioNormalCdf(SN03_tau_2)-bioNormalCdf(SN03_tau_1),
    3: bioNormalCdf(SN03_tau_3)-bioNormalCdf(SN03_tau_2),
    4: bioNormalCdf(SN03_tau_4)-bioNormalCdf(SN03_tau_3),
    5: bioNormalCdf(SN03_tau_5)-bioNormalCdf(SN03_tau_4),
    6: 1-bioNormalCdf(SN03_tau_5)
}
P_SN03 = Elem(IndSN03, socialnorm4) 


In [29]:
B_EP01_F1 = Beta('B_EP01_F1',-1,None,None,1)
B_EP02_F1 = Beta('B_EP02_F1',-1,None,None,0)
B_EP03_F1 = Beta('B_EP03_F1',-1,None,None,0)
B_EP04_F1 = Beta('B_EP04_F1',-1,None,None,0)
B_EP05_F1 = Beta('B_EP05_F1',-1,None,None,0)

INTER_EP01 = Beta('INTER_EP01',0,None,None,1)
INTER_EP02 = Beta('INTER_EP02',0,None,None,0)
INTER_EP03 = Beta('INTER_EP03',0,None,None,0)
INTER_EP04 = Beta('INTER_EP04',0,None,None,0)
INTER_EP05 = Beta('INTER_EP05',0,None,None,0)

MODEL_EP01 = INTER_EP01 + B_EP01_F1 * EP
MODEL_EP02 = INTER_EP02 + B_EP02_F1 * EP
MODEL_EP03 = INTER_EP03 + B_EP03_F1 * EP
MODEL_EP04 = INTER_EP04 + B_EP04_F1 * EP
MODEL_EP05 = INTER_EP05 + B_EP05_F1 * EP

SIGMA_STAR_EP01 = Beta('SIGMA_STAR_EP01',1,1.0e-5,None,1)
SIGMA_STAR_EP02 = Beta('SIGMA_STAR_EP02',1,1.0e-5,None,0)
SIGMA_STAR_EP03 = Beta('SIGMA_STAR_EP03',1,1.0e-5,None,0)
SIGMA_STAR_EP04 = Beta('SIGMA_STAR_EP04',1,1.0e-5,None,0)
SIGMA_STAR_EP05 = Beta('SIGMA_STAR_EP05',1,1.0e-5,None,0)

EP01_tau_1 = (tau_1-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_2 = (tau_2-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_3 = (tau_3-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_4 = (tau_4-MODEL_EP01)/SIGMA_STAR_EP01
EP01_tau_5 = (tau_5-MODEL_EP01)/SIGMA_STAR_EP01

EP02_tau_1 = (tau_1-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_2 = (tau_2-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_3 = (tau_3-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_4 = (tau_4-MODEL_EP02)/SIGMA_STAR_EP02
EP02_tau_5 = (tau_5-MODEL_EP02)/SIGMA_STAR_EP02

EP03_tau_1 = (tau_1-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_2 = (tau_2-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_3 = (tau_3-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_4 = (tau_4-MODEL_EP03)/SIGMA_STAR_EP03
EP03_tau_5 = (tau_5-MODEL_EP03)/SIGMA_STAR_EP03

EP04_tau_1 = (tau_1-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_2 = (tau_2-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_3 = (tau_3-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_4 = (tau_4-MODEL_EP04)/SIGMA_STAR_EP04
EP04_tau_5 = (tau_5-MODEL_EP04)/SIGMA_STAR_EP04

EP05_tau_1 = (tau_1-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_2 = (tau_2-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_3 = (tau_3-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_4 = (tau_4-MODEL_EP05)/SIGMA_STAR_EP05
EP05_tau_5 = (tau_5-MODEL_EP05)/SIGMA_STAR_EP05

IndEP01 = {
    1: bioNormalCdf(EP01_tau_1),
    2: bioNormalCdf(EP01_tau_2)-bioNormalCdf(EP01_tau_1),
    3: bioNormalCdf(EP01_tau_3)-bioNormalCdf(EP01_tau_2),
    4: bioNormalCdf(EP01_tau_4)-bioNormalCdf(EP01_tau_3),
    5: bioNormalCdf(EP01_tau_5)-bioNormalCdf(EP01_tau_4),
    6: 1-bioNormalCdf(EP01_tau_5)
}
P_EP01 = Elem(IndEP01, expectedperf1)


IndEP02 = {
    1: bioNormalCdf(EP02_tau_1),
    2: bioNormalCdf(EP02_tau_2)-bioNormalCdf(EP02_tau_1),
    3: bioNormalCdf(EP02_tau_3)-bioNormalCdf(EP02_tau_2),
    4: bioNormalCdf(EP02_tau_4)-bioNormalCdf(EP02_tau_3),
    5: bioNormalCdf(EP02_tau_5)-bioNormalCdf(EP02_tau_4),
    6: 1-bioNormalCdf(EP02_tau_5)
}
P_EP02 = Elem(IndEP02, expectedperf2)

IndEP03 = {
    1: bioNormalCdf(EP03_tau_1),
    2: bioNormalCdf(EP03_tau_2)-bioNormalCdf(EP03_tau_1),
    3: bioNormalCdf(EP03_tau_3)-bioNormalCdf(EP03_tau_2),
    4: bioNormalCdf(EP03_tau_4)-bioNormalCdf(EP03_tau_3),
    5: bioNormalCdf(EP03_tau_5)-bioNormalCdf(EP03_tau_4),
    6: 1-bioNormalCdf(EP03_tau_5)
}
P_EP03 = Elem(IndEP03, expectedperf3) 

IndEP04 = {
    1: bioNormalCdf(EP04_tau_1),
    2: bioNormalCdf(EP04_tau_2)-bioNormalCdf(EP04_tau_1),
    3: bioNormalCdf(EP04_tau_3)-bioNormalCdf(EP04_tau_2),
    4: bioNormalCdf(EP04_tau_4)-bioNormalCdf(EP04_tau_3),
    5: bioNormalCdf(EP04_tau_5)-bioNormalCdf(EP04_tau_4),
    6: 1-bioNormalCdf(EP04_tau_5)
}
P_EP04 = Elem(IndEP04, expectedperf4) 

IndEP05 = {
    1: bioNormalCdf(EP05_tau_1),
    2: bioNormalCdf(EP05_tau_2)-bioNormalCdf(EP05_tau_1),
    3: bioNormalCdf(EP05_tau_3)-bioNormalCdf(EP05_tau_2),
    4: bioNormalCdf(EP05_tau_4)-bioNormalCdf(EP05_tau_3),
    5: bioNormalCdf(EP05_tau_5)-bioNormalCdf(EP05_tau_4),
    6: 1-bioNormalCdf(EP05_tau_5)
}
P_EP05 = Elem(IndEP05, expectedperf5) 

In [23]:
# B_RISK01_F1 = Beta('B_RISK01_F1',-1,None,None,1)
# B_RISK02_F1 = Beta('B_RISK02_F1',-1,None,None,0)
# B_RISK03_F1 = Beta('B_RISK03_F1',-1,None,None,0)

# INTER_RISK01 = Beta('INTER_RISK01',0,None,None,1)
# INTER_RISK02 = Beta('INTER_RISK02',0,None,None,0)
# INTER_RISK03 = Beta('INTER_RISK03',0,None,None,0)

# MODEL_RISK01 = INTER_RISK01 + B_RISK01_F1 * RISK
# MODEL_RISK02 = INTER_RISK02 + B_RISK02_F1 * RISK
# MODEL_RISK03 = INTER_RISK03 + B_RISK03_F1 * RISK

# SIGMA_STAR_RISK01 = Beta('SIGMA_STAR_RISK01',1,1.0e-5,None,1)
# SIGMA_STAR_RISK02 = Beta('SIGMA_STAR_RISK02',1,1.0e-5,None,0)
# SIGMA_STAR_RISK03 = Beta('SIGMA_STAR_RISK03',1,1.0e-5,None,0)

# RISK01_tau_1 = (tau_1-MODEL_RISK01)/SIGMA_STAR_RISK01
# RISK01_tau_2 = (tau_2-MODEL_RISK01)/SIGMA_STAR_RISK01
# RISK01_tau_3 = (tau_3-MODEL_RISK01)/SIGMA_STAR_RISK01
# RISK01_tau_4 = (tau_4-MODEL_RISK01)/SIGMA_STAR_RISK01
# RISK01_tau_5 = (tau_5-MODEL_RISK01)/SIGMA_STAR_RISK01

# RISK02_tau_1 = (tau_1-MODEL_RISK02)/SIGMA_STAR_RISK02
# RISK02_tau_2 = (tau_2-MODEL_RISK02)/SIGMA_STAR_RISK02
# RISK02_tau_3 = (tau_3-MODEL_RISK02)/SIGMA_STAR_RISK02
# RISK02_tau_4 = (tau_4-MODEL_RISK02)/SIGMA_STAR_RISK02
# RISK02_tau_5 = (tau_5-MODEL_RISK02)/SIGMA_STAR_RISK02

# RISK03_tau_1 = (tau_1-MODEL_RISK03)/SIGMA_STAR_RISK03
# RISK03_tau_2 = (tau_2-MODEL_RISK03)/SIGMA_STAR_RISK03
# RISK03_tau_3 = (tau_3-MODEL_RISK03)/SIGMA_STAR_RISK03
# RISK03_tau_4 = (tau_4-MODEL_RISK03)/SIGMA_STAR_RISK03
# RISK03_tau_5 = (tau_5-MODEL_RISK03)/SIGMA_STAR_RISK03


# # Likert scale with M = 6 measurements
# IndRISK01 = {
#     1: bioNormalCdf(RISK01_tau_1),
#     2: bioNormalCdf(RISK01_tau_2)-bioNormalCdf(RISK01_tau_1),
#     3: bioNormalCdf(RISK01_tau_3)-bioNormalCdf(RISK01_tau_2),
#     4: bioNormalCdf(RISK01_tau_4)-bioNormalCdf(RISK01_tau_3),
#     5: bioNormalCdf(RISK01_tau_5)-bioNormalCdf(RISK01_tau_4),
#     6: 1-bioNormalCdf(RISK01_tau_5)
# }
# P_RISK01 = Elem(IndRISK01, perceivedrisk1)

# IndRISK02 = {
#     1: bioNormalCdf(RISK02_tau_1),
#     2: bioNormalCdf(RISK02_tau_2)-bioNormalCdf(RISK02_tau_1),
#     3: bioNormalCdf(RISK02_tau_3)-bioNormalCdf(RISK02_tau_2),
#     4: bioNormalCdf(RISK02_tau_4)-bioNormalCdf(RISK02_tau_3),
#     5: bioNormalCdf(RISK02_tau_5)-bioNormalCdf(RISK02_tau_4),
#     6: 1-bioNormalCdf(RISK02_tau_5)
# }
# P_RISK02 = Elem(IndRISK02, perceivedrisk2)

# IndRISK03 = {
#     1: bioNormalCdf(RISK03_tau_1),
#     2: bioNormalCdf(RISK03_tau_2)-bioNormalCdf(RISK03_tau_1),
#     3: bioNormalCdf(RISK03_tau_3)-bioNormalCdf(RISK03_tau_2),
#     4: bioNormalCdf(RISK03_tau_4)-bioNormalCdf(RISK03_tau_3),
#     5: bioNormalCdf(RISK03_tau_5)-bioNormalCdf(RISK03_tau_4),
#     6: 1-bioNormalCdf(RISK03_tau_5)
# }
# P_RISK03 = Elem(IndRISK03, perceivedrisk3)


In [30]:
B_EB01_F1 = Beta('B_EB01_F1',-1,None,None,1)
B_EB02_F1 = Beta('B_EB02_F1',-1,None,None,0)
B_EB03_F1 = Beta('B_EB03_F1',-1,None,None,0)

INTER_EB01 = Beta('INTER_EB01',0,None,None,1)
INTER_EB02 = Beta('INTER_EB02',0,None,None,0)
INTER_EB03 = Beta('INTER_EB03',0,None,None,0)

MODEL_EB01 = INTER_EB01 + B_EB01_F1 * EB
MODEL_EB02 = INTER_EB02 + B_EB02_F1 * EB
MODEL_EB03 = INTER_EB03 + B_EB03_F1 * EB

SIGMA_STAR_EB01 = Beta('SIGMA_STAR_EB01',1,1.0e-5,None,1)
SIGMA_STAR_EB02 = Beta('SIGMA_STAR_EB02',1,1.0e-5,None,0)
SIGMA_STAR_EB03 = Beta('SIGMA_STAR_EB03',1,1.0e-5,None,0)

EB01_tau_1 = (tau_1-MODEL_EB01)/SIGMA_STAR_EB01
EB01_tau_2 = (tau_2-MODEL_EB01)/SIGMA_STAR_EB01
EB01_tau_3 = (tau_3-MODEL_EB01)/SIGMA_STAR_EB01
EB01_tau_4 = (tau_4-MODEL_EB01)/SIGMA_STAR_EB01
EB01_tau_5 = (tau_5-MODEL_EB01)/SIGMA_STAR_EB01

EB02_tau_1 = (tau_1-MODEL_EB02)/SIGMA_STAR_EB02
EB02_tau_2 = (tau_2-MODEL_EB02)/SIGMA_STAR_EB02
EB02_tau_3 = (tau_3-MODEL_EB02)/SIGMA_STAR_EB02
EB02_tau_4 = (tau_4-MODEL_EB02)/SIGMA_STAR_EB02
EB02_tau_5 = (tau_5-MODEL_EB02)/SIGMA_STAR_EB02

EB03_tau_1 = (tau_1-MODEL_EB03)/SIGMA_STAR_EB03
EB03_tau_2 = (tau_2-MODEL_EB03)/SIGMA_STAR_EB03
EB03_tau_3 = (tau_3-MODEL_EB03)/SIGMA_STAR_EB03
EB03_tau_4 = (tau_4-MODEL_EB03)/SIGMA_STAR_EB03
EB03_tau_5 = (tau_5-MODEL_EB03)/SIGMA_STAR_EB03

# Likert scale with M = 6 measurements
IndEB01 = {
    1: bioNormalCdf(EB01_tau_1),
    2: bioNormalCdf(EB01_tau_2)-bioNormalCdf(EB01_tau_1),
    3: bioNormalCdf(EB01_tau_3)-bioNormalCdf(EB01_tau_2),
    4: bioNormalCdf(EB01_tau_4)-bioNormalCdf(EB01_tau_3),
    5: bioNormalCdf(EB01_tau_5)-bioNormalCdf(EB01_tau_4),
    6: 1-bioNormalCdf(EB01_tau_5)
}
P_EB01 = Elem(IndEB01, envbenefit1)

IndEB02 = {
    1: bioNormalCdf(EB02_tau_1),
    2: bioNormalCdf(EB02_tau_2)-bioNormalCdf(EB02_tau_1),
    3: bioNormalCdf(EB02_tau_3)-bioNormalCdf(EB02_tau_2),
    4: bioNormalCdf(EB02_tau_4)-bioNormalCdf(EB02_tau_3),
    5: bioNormalCdf(EB02_tau_5)-bioNormalCdf(EB02_tau_4),
    6: 1-bioNormalCdf(EB02_tau_5)
}
P_EB02 = Elem(IndEB02, envbenefit2)

IndEB03 = {
    1: bioNormalCdf(EB03_tau_1),
    2: bioNormalCdf(EB03_tau_2)-bioNormalCdf(EB03_tau_1),
    3: bioNormalCdf(EB03_tau_3)-bioNormalCdf(EB03_tau_2),
    4: bioNormalCdf(EB03_tau_4)-bioNormalCdf(EB03_tau_3),
    5: bioNormalCdf(EB03_tau_5)-bioNormalCdf(EB03_tau_4),
    6: 1-bioNormalCdf(EB03_tau_5)
}
P_EB03 = Elem(IndEB03, envbenefit3)


In [31]:
loglike = (
    log(P_SN01)
    + log(P_SN02)
    + log(P_SN03)
    + log(P_EP01)
    + log(P_EP02)
    + log(P_EP03)
    + log(P_EP04)
    + log(P_EP05)
    # + log(P_RISK01)
    # + log(P_RISK02)
    # + log(P_RISK03)
    + log(P_EB01)
    + log(P_EB02)
    + log(P_EB03)
)

the_biogeme = bio.BIOGEME(database, loglike)
the_biogeme.modelName = 'b02_ordered_latent_SN_EP_EB'

results = the_biogeme.estimate()

[17:30:53] < Warning >   Cannot read file __b02_ordered_latent_SN_EP_EB.iter. Statement is ignored.


In [26]:
print(f'Estimated betas: {len(results.data.betaValues)}')
print(f'final log likelihood: {results.data.logLike:.3f}')
print(f'Output file: {results.data.htmlFileName}')
results.writeLaTeX()
print(f'LaTeX file: {results.data.latexFileName}')

Estimated betas: 54
final log likelihood: -86352.728
Output file: b02_ordered_latent.html
LaTeX file: b02_ordered_latent.tex
